In [1]:
from PExp import *

## Experiment: 1 die

In [2]:
die = PExp(6)
die.info()

----------------------------------------------------------------------------------------------------
Population: {('4',): 1, ('6',): 1, ('3',): 1, ('5',): 1, ('2',): 1, ('1',): 1}
Alphabet: [('1',), ('2',), ('3',), ('4',), ('5',), ('6',)]
Prior_prob: [ 0.16666667  0.16666667  0.16666667  0.16666667  0.16666667  0.16666667]
Prior_prob_frac[1]: 1/6
Prior_prob_frac[2]: 1/6
Prior_prob_frac[3]: 1/6
Prior_prob_frac[4]: 1/6
Prior_prob_frac[5]: 1/6
Prior_prob_frac[6]: 1/6
N_outcomes: 6
N_trials: None
Post_prob: [ 0.  0.  0.  0.  0.  0.]
Recursive exp: False
Exp_dict:
{('1',): {'post_prob': 0.0,
          'prior_prob': 0.16666666666666666,
          'prior_prob_frac': Fraction(1, 6)},
 ('2',): {'post_prob': 0.0,
          'prior_prob': 0.16666666666666666,
          'prior_prob_frac': Fraction(1, 6)},
 ('3',): {'post_prob': 0.0,
          'prior_prob': 0.16666666666666666,
          'prior_prob_frac': Fraction(1, 6)},
 ('4',): {'post_prob': 0.0,
          'prior_prob': 0.16666666666666666,
    

### Event: even numbers greater than 3 (4,6)

In [3]:
evEven = Event([2,4,6],die)
evGr3 = Event([4,5,6],die)
evEvenAndGr3 = evEven * evGr3

In [4]:
evEvenAndGr3.print_prob()

Probability: 0.333333333333 ( 1/3 )


In [38]:
evEven.is_independent(evGr3)

False

### Event: either even numbers or greater than 3 (2,4,5,6)

In [5]:
evEvenOrGr3 = evEven + evGr3

In [6]:
evEvenOrGr3.print_prob()

Probability: 0.666666666667 ( 2/3 )


In [7]:
evEven.exp.alphabet

[('1',), ('2',), ('3',), ('4',), ('5',), ('6',)]

## Experiment: family with 3 sons

In [8]:
fam1 = PExp(['M','F'])
fam2 = PExp(['M','F'])
fam3 = PExp(['M','F'])
fam = fam1*fam2*fam3
fam.info()

----------------------------------------------------------------------------------------------------
Population: {('M', 'F', 'M'): 1, ('F', 'F', 'M'): 1, ('F', 'M', 'M'): 1, ('F', 'M', 'F'): 1, ('F', 'F', 'F'): 1, ('M', 'M', 'F'): 1, ('M', 'M', 'M'): 1, ('M', 'F', 'F'): 1}
Alphabet: [('M', 'M', 'M'), ('M', 'M', 'F'), ('M', 'F', 'M'), ('M', 'F', 'F'), ('F', 'M', 'M'), ('F', 'M', 'F'), ('F', 'F', 'M'), ('F', 'F', 'F')]
Prior_prob: [ 0.125  0.125  0.125  0.125  0.125  0.125  0.125  0.125]
Prior_prob_frac[1]: 1/8
Prior_prob_frac[2]: 1/8
Prior_prob_frac[3]: 1/8
Prior_prob_frac[4]: 1/8
Prior_prob_frac[5]: 1/8
Prior_prob_frac[6]: 1/8
Prior_prob_frac[7]: 1/8
Prior_prob_frac[8]: 1/8
N_outcomes: 8
N_trials: None
Post_prob: [ 0.  0.  0.  0.  0.  0.  0.  0.]
Recursive exp: False
Exp_dict:
{('F', 'F', 'F'): {'post_prob': 0.0,
                   'prior_prob': 0.125,
                   'prior_prob_frac': Fraction(1, 8)},
 ('F', 'F', 'M'): {'post_prob': 0.0,
                   'prior_prob': 0.125,
   

### Probability of 3M given that there is at least 1M

In [9]:
ev1M = Event([('M','*','*')],fam)
ev2M = Event([('*','M','*')],fam)
ev3M = Event([('*','*','M')],fam)
evAtLeast1M = ev1M + ev2M + ev3M

In [10]:
ev3M = Event([('M','M','M')],fam)
ev3MatLeast1M = ev3M|evAtLeast1M

In [11]:
ev3M.print_prob()

Probability: 0.125 ( 1/8 )


## Experiment: 1 card

In [12]:
exp_cards = PExp(alphabet=['A','2','3','4','5','6','7','8','9','10','J','Q','K'])
exp_suits = PExp(alphabet=['♥','♦','♠','♣'])
exp_deck = exp_cards * exp_suits
exp_deck.info()

----------------------------------------------------------------------------------------------------
Population: {('2', '♥'): 1, ('6', '♥'): 1, ('8', '♠'): 1, ('3', '♦'): 1, ('5', '♣'): 1, ('7', '♠'): 1, ('A', '♥'): 1, ('7', '♦'): 1, ('6', '♠'): 1, ('Q', '♦'): 1, ('8', '♦'): 1, ('9', '♠'): 1, ('K', '♥'): 1, ('6', '♣'): 1, ('K', '♦'): 1, ('5', '♦'): 1, ('2', '♦'): 1, ('6', '♦'): 1, ('8', '♥'): 1, ('7', '♣'): 1, ('J', '♥'): 1, ('Q', '♣'): 1, ('J', '♦'): 1, ('3', '♣'): 1, ('10', '♥'): 1, ('A', '♠'): 1, ('2', '♣'): 1, ('5', '♥'): 1, ('4', '♠'): 1, ('5', '♠'): 1, ('K', '♠'): 1, ('4', '♥'): 1, ('J', '♣'): 1, ('9', '♣'): 1, ('4', '♦'): 1, ('10', '♠'): 1, ('A', '♦'): 1, ('K', '♣'): 1, ('10', '♣'): 1, ('J', '♠'): 1, ('2', '♠'): 1, ('10', '♦'): 1, ('Q', '♥'): 1, ('3', '♥'): 1, ('3', '♠'): 1, ('4', '♣'): 1, ('7', '♥'): 1, ('9', '♥'): 1, ('A', '♣'): 1, ('Q', '♠'): 1, ('9', '♦'): 1, ('8', '♣'): 1}
Alphabet: [('A', '♥'), ('A', '♦'), ('A', '♠'), ('A', '♣'), ('2', '♥'), ('2', '♦'), ('2', '♠'), ('2', '

### Event 1: Heart

In [13]:
e1 = Event([('*','♥')],exp_deck)
print(e1)
e1.print_prob()

[('A', '♥'), ('2', '♥'), ('3', '♥'), ('4', '♥'), ('5', '♥'), ('6', '♥'), ('7', '♥'), ('8', '♥'), ('9', '♥'), ('10', '♥'), ('J', '♥'), ('Q', '♥'), ('K', '♥')]: 0.25
Probability: 0.25 ( 1/4 )


### Event 2: Jack

In [14]:
e2 = Event([('J','*')],exp_deck)
print(e2)
e2.print_prob()

[('J', '♥'), ('J', '♦'), ('J', '♠'), ('J', '♣')]: 0.0769230769231
Probability: 0.0769230769231 ( 1/13 )


### Event 3: Jack given that a Heart card is withdrawed

In [15]:
e3 = e2|e1
print(e3)
e3.print_prob()

[('J', '♥')]: 0.0769230769231
Probability: 0.0769230769231 ( 1/13 )


### Event 4: Heart given that a Jack is withdrawed

In [16]:
e4 = e1|e2
print(e4)
e4.print_prob()

[('J', '♥')]: 0.25
Probability: 0.25 ( 1/4 )


## Experiment: 5 cards

In [51]:
exp_deck1 = exp_cards * exp_suits
exp_deck1.set_recursive(True)
exp_deck1.info()

----------------------------------------------------------------------------------------------------
Population: {('2', '♥'): 1, ('6', '♥'): 1, ('8', '♠'): 1, ('3', '♦'): 1, ('5', '♣'): 1, ('7', '♠'): 1, ('A', '♥'): 1, ('7', '♦'): 1, ('6', '♠'): 1, ('Q', '♦'): 1, ('8', '♦'): 1, ('9', '♠'): 1, ('K', '♥'): 1, ('6', '♣'): 1, ('K', '♦'): 1, ('5', '♦'): 1, ('2', '♦'): 1, ('6', '♦'): 1, ('8', '♥'): 1, ('7', '♣'): 1, ('J', '♥'): 1, ('Q', '♣'): 1, ('J', '♦'): 1, ('3', '♣'): 1, ('10', '♥'): 1, ('A', '♠'): 1, ('2', '♣'): 1, ('5', '♥'): 1, ('4', '♠'): 1, ('5', '♠'): 1, ('K', '♠'): 1, ('4', '♥'): 1, ('J', '♣'): 1, ('9', '♣'): 1, ('4', '♦'): 1, ('10', '♠'): 1, ('A', '♦'): 1, ('K', '♣'): 1, ('10', '♣'): 1, ('J', '♠'): 1, ('2', '♠'): 1, ('10', '♦'): 1, ('Q', '♥'): 1, ('3', '♥'): 1, ('3', '♠'): 1, ('4', '♣'): 1, ('7', '♥'): 1, ('9', '♥'): 1, ('A', '♣'): 1, ('Q', '♠'): 1, ('9', '♦'): 1, ('8', '♣'): 1}
Alphabet: [('A', '♥'), ('A', '♦'), ('A', '♠'), ('A', '♣'), ('2', '♥'), ('2', '♦'), ('2', '♠'), ('2', '

### First card is Ace of Hearts

In [52]:
e1 = Event([('A','♥')],exp_deck1)
print(e1)
e1.print_prob()

[('A', '♥')]: 0.0192307692308
Probability: 0.0192307692308 ( 1/52 )


### Second card is Ace of Diamonds

In [53]:
e2 = Event([('A','♦')],exp_deck1)
print(e2)
e2.print_prob()

[('A', '♦')]: 0.0196078431373
Probability: 0.0196078431373 ( 1/51 )


### Third card is Ace of Spades

In [54]:
e3 = Event([('A','♠')],exp_deck1)
print(e3)
e3.print_prob()

[('A', '♠')]: 0.02
Probability: 0.02 ( 1/50 )


### Fourth card is Ace of Clubs

In [55]:
e4 = Event([('A','♣')],exp_deck1)
print(e4)
e4.print_prob()

[('A', '♣')]: 0.0204081632653
Probability: 0.0204081632653 ( 1/49 )


### Probability of Aces Poker

In [56]:
e1.prob * e2.prob * e3.prob * e4.prob * 5* 4 * 3 * 2 * 1

1.8468926031951241e-05

## Experiment: box with black and white balls (recursive)

In [23]:
expBW = PExp(population={'W':75,'B':25},rec=True)
expBW.info()

----------------------------------------------------------------------------------------------------
Population: {('B',): 25, ('W',): 75}
Alphabet: [('W',), ('B',)]
Prior_prob: [ 0.75  0.25]
Prior_prob_frac[1]: 3/4
Prior_prob_frac[2]: 1/4
N_outcomes: 2
N_trials: None
Post_prob: [ 0.  0.]
Recursive exp: True
Exp_dict:
{('B',): {'post_prob': 0.0,
          'prior_prob': 0.25,
          'prior_prob_frac': Fraction(1, 4)},
 ('W',): {'post_prob': 0.0,
          'prior_prob': 0.75,
          'prior_prob_frac': Fraction(3, 4)}}



### Event: 1 white ball

In [24]:
eW = Event([('W',)],expBW)
print(eW)
eW.print_prob()

[('W',)]: 0.75
Probability: 0.75 ( 3/4 )


In [25]:
expBW.info()

----------------------------------------------------------------------------------------------------
Population: {('B',): 25, ('W',): 74}
Alphabet: [('W',), ('B',)]
Prior_prob: [ 0.74747475  0.25252525]
Prior_prob_frac[1]: 74/99
Prior_prob_frac[2]: 25/99
N_outcomes: 2
N_trials: None
Post_prob: [ 0.  0.]
Recursive exp: True
Exp_dict:
{('B',): {'post_prob': 0.0,
          'prior_prob': 0.25252525252525254,
          'prior_prob_frac': Fraction(25, 99)},
 ('W',): {'post_prob': 0.0,
          'prior_prob': 0.74747474747474751,
          'prior_prob_frac': Fraction(74, 99)}}



### Event: 1 black ball

In [26]:
expBW.reset_exp()
expBW.info()
eB = Event(['B'],expBW)
print(eB)
eB.print_prob()

----------------------------------------------------------------------------------------------------
Population: {('B',): 25, ('W',): 75}
Alphabet: [('W',), ('B',)]
Prior_prob: [ 0.75  0.25]
Prior_prob_frac[1]: 3/4
Prior_prob_frac[2]: 1/4
N_outcomes: 2
N_trials: None
Post_prob: [ 0.  0.]
Recursive exp: True
Exp_dict:
{('B',): {'post_prob': 0.0,
          'prior_prob': 0.25,
          'prior_prob_frac': Fraction(1, 4)},
 ('W',): {'post_prob': 0.0,
          'prior_prob': 0.75,
          'prior_prob_frac': Fraction(3, 4)}}

[('B',)]: 0.25
Probability: 0.25 ( 1/4 )


In [27]:
expBW.info()

----------------------------------------------------------------------------------------------------
Population: {('B',): 24, ('W',): 75}
Alphabet: [('W',), ('B',)]
Prior_prob: [ 0.75757576  0.24242424]
Prior_prob_frac[1]: 25/33
Prior_prob_frac[2]: 8/33
N_outcomes: 2
N_trials: None
Post_prob: [ 0.  0.]
Recursive exp: True
Exp_dict:
{('B',): {'post_prob': 0.0,
          'prior_prob': 0.24242424242424243,
          'prior_prob_frac': Fraction(8, 33)},
 ('W',): {'post_prob': 0.0,
          'prior_prob': 0.75757575757575757,
          'prior_prob_frac': Fraction(25, 33)}}



### Event: 1 white ball and 1 black ball

In [28]:
expBW.reset_exp()
eW = Event([('W',)],expBW)
print('Probability W:',eW.prob,'(',Fraction(eW.prob).limit_denominator(1000),')')
expBW.info()
print('\n\n')
eB = Event([('B',)],expBW)
print('Probability B:',eB.prob,'(',Fraction(eB.prob).limit_denominator(1000),')')
expBW.info()
print('\n\n')
print('Probability WB:',eW.prob * eB.prob,'(',Fraction(eW.prob * eB.prob).limit_denominator(1000),')')

Probability W: 0.75 ( 3/4 )
----------------------------------------------------------------------------------------------------
Population: {('B',): 25, ('W',): 74}
Alphabet: [('W',), ('B',)]
Prior_prob: [ 0.74747475  0.25252525]
Prior_prob_frac[1]: 74/99
Prior_prob_frac[2]: 25/99
N_outcomes: 2
N_trials: None
Post_prob: [ 0.  0.]
Recursive exp: True
Exp_dict:
{('B',): {'post_prob': 0.0,
          'prior_prob': 0.25252525252525254,
          'prior_prob_frac': Fraction(25, 99)},
 ('W',): {'post_prob': 0.0,
          'prior_prob': 0.74747474747474751,
          'prior_prob_frac': Fraction(74, 99)}}




Probability B: 0.252525252525 ( 25/99 )
----------------------------------------------------------------------------------------------------
Population: {('B',): 24, ('W',): 74}
Alphabet: [('W',), ('B',)]
Prior_prob: [ 0.75510204  0.24489796]
Prior_prob_frac[1]: 37/49
Prior_prob_frac[2]: 12/49
N_outcomes: 2
N_trials: None
Post_prob: [ 0.  0.]
Recursive exp: True
Exp_dict:
{('B',): {'post_

## Experiment: 3 siblings

In [30]:
sib = PExp(['M','F'])
sib3 = sib * sib * sib
sib3.info()

----------------------------------------------------------------------------------------------------
Population: {('M', 'F', 'M'): 1, ('F', 'F', 'M'): 1, ('F', 'M', 'M'): 1, ('F', 'M', 'F'): 1, ('F', 'F', 'F'): 1, ('M', 'M', 'F'): 1, ('M', 'M', 'M'): 1, ('M', 'F', 'F'): 1}
Alphabet: [('M', 'M', 'M'), ('M', 'M', 'F'), ('M', 'F', 'M'), ('M', 'F', 'F'), ('F', 'M', 'M'), ('F', 'M', 'F'), ('F', 'F', 'M'), ('F', 'F', 'F')]
Prior_prob: [ 0.125  0.125  0.125  0.125  0.125  0.125  0.125  0.125]
Prior_prob_frac[1]: 1/8
Prior_prob_frac[2]: 1/8
Prior_prob_frac[3]: 1/8
Prior_prob_frac[4]: 1/8
Prior_prob_frac[5]: 1/8
Prior_prob_frac[6]: 1/8
Prior_prob_frac[7]: 1/8
Prior_prob_frac[8]: 1/8
N_outcomes: 8
N_trials: None
Post_prob: [ 0.  0.  0.  0.  0.  0.  0.  0.]
Recursive exp: False
Exp_dict:
{('F', 'F', 'F'): {'post_prob': 0.0,
                   'prior_prob': 0.125,
                   'prior_prob_frac': Fraction(1, 8)},
 ('F', 'F', 'M'): {'post_prob': 0.0,
                   'prior_prob': 0.125,
   

## Event 1: the first sibling is M

In [31]:
ev1 = Event([('M','*','*')],sib3)
print(ev1)
ev1.print_prob()

[('M', 'M', 'M'), ('M', 'M', 'F'), ('M', 'F', 'M'), ('M', 'F', 'F')]: 0.5
Probability: 0.5 ( 1/2 )


## Event 2: 3 Males

In [32]:
ev2 = Event([('M','M','M')],sib3)
print(ev2)
ev2.print_prob()

[('M', 'M', 'M')]: 0.125
Probability: 0.125 ( 1/8 )


## Event 3: at least one sibling is M

In [33]:
ev3 = Event([('M','*','*'),('*','M','*'),('*','*','M')],sib3)
print(ev3)
ev3.print_prob()

[('M', 'M', 'M'), ('M', 'M', 'F'), ('M', 'F', 'M'), ('M', 'F', 'F'), ('M', 'M', 'M'), ('M', 'M', 'F'), ('M', 'M', 'M')]: 0.875
Probability: 0.875 ( 7/8 )


### P(ev2|ev1)

In [34]:
ev2GivenEv1 = ev2|ev1
print(ev2GivenEv1)
ev2GivenEv1.print_prob()

[('M', 'M', 'M')]: 0.25
Probability: 0.25 ( 1/4 )


### P(ev2|ev3)

In [35]:
ev2GivenEv3 = ev2|ev3
print(ev2GivenEv3)
ev2GivenEv3.print_prob()

[('M', 'M', 'M')]: 0.142857142857
Probability: 0.142857142857 ( 1/7 )


In [36]:
ev2.is_independent(ev1)

False

In [37]:
ev2.is_independent(ev3)

False

## Experiment: box with 18 numbered balls

In [43]:
box18 = PExp(18)
#box18.set_recursive(True)
box18.info()

----------------------------------------------------------------------------------------------------
Population: {('6',): 1, ('10',): 1, ('3',): 1, ('5',): 1, ('15',): 1, ('8',): 1, ('17',): 1, ('12',): 1, ('18',): 1, ('4',): 1, ('13',): 1, ('9',): 1, ('16',): 1, ('14',): 1, ('11',): 1, ('2',): 1, ('7',): 1, ('1',): 1}
Alphabet: [('1',), ('2',), ('3',), ('4',), ('5',), ('6',), ('7',), ('8',), ('9',), ('10',), ('11',), ('12',), ('13',), ('14',), ('15',), ('16',), ('17',), ('18',)]
Prior_prob: [ 0.05555556  0.05555556  0.05555556  0.05555556  0.05555556  0.05555556
  0.05555556  0.05555556  0.05555556  0.05555556  0.05555556  0.05555556
  0.05555556  0.05555556  0.05555556  0.05555556  0.05555556  0.05555556]
Prior_prob_frac[1]: 1/18
Prior_prob_frac[2]: 1/18
Prior_prob_frac[3]: 1/18
Prior_prob_frac[4]: 1/18
Prior_prob_frac[5]: 1/18
Prior_prob_frac[6]: 1/18
Prior_prob_frac[7]: 1/18
Prior_prob_frac[8]: 1/18
Prior_prob_frac[9]: 1/18
Prior_prob_frac[10]: 1/18
Prior_prob_frac[11]: 1/18
Prior_

### Event: the extracted ball is multiple of 2

In [44]:
evMul2 = Event([2,4,6,8,10,12,14,16,18],box18)
print(evMul2)
evMul2.print_prob()

[('2',), ('4',), ('6',), ('8',), ('10',), ('12',), ('14',), ('16',), ('18',)]: 0.5
Probability: 0.5 ( 1/2 )


### Event: the extracted ball is multiple of 3

In [45]:
evMul3 = Event([3,6,9,12,15,18],box18)
print(evMul3)
evMul3.print_prob()

[('3',), ('6',), ('9',), ('12',), ('15',), ('18',)]: 0.333333333333
Probability: 0.333333333333 ( 1/3 )


In [46]:
evMul2.is_independent(evMul3)

True

## Experiment: box with 21 numbered balls

In [47]:
box21 = PExp(21)
#box18.set_recursive(True)
box21.info()

----------------------------------------------------------------------------------------------------
Population: {('6',): 1, ('10',): 1, ('3',): 1, ('5',): 1, ('15',): 1, ('8',): 1, ('20',): 1, ('17',): 1, ('19',): 1, ('12',): 1, ('18',): 1, ('4',): 1, ('13',): 1, ('9',): 1, ('16',): 1, ('14',): 1, ('11',): 1, ('2',): 1, ('7',): 1, ('21',): 1, ('1',): 1}
Alphabet: [('1',), ('2',), ('3',), ('4',), ('5',), ('6',), ('7',), ('8',), ('9',), ('10',), ('11',), ('12',), ('13',), ('14',), ('15',), ('16',), ('17',), ('18',), ('19',), ('20',), ('21',)]
Prior_prob: [ 0.04761905  0.04761905  0.04761905  0.04761905  0.04761905  0.04761905
  0.04761905  0.04761905  0.04761905  0.04761905  0.04761905  0.04761905
  0.04761905  0.04761905  0.04761905  0.04761905  0.04761905  0.04761905
  0.04761905  0.04761905  0.04761905]
Prior_prob_frac[1]: 1/21
Prior_prob_frac[2]: 1/21
Prior_prob_frac[3]: 1/21
Prior_prob_frac[4]: 1/21
Prior_prob_frac[5]: 1/21
Prior_prob_frac[6]: 1/21
Prior_prob_frac[7]: 1/21
Prior_pr

### Event: the extracted ball is multiple of 2

In [48]:
evMul2 = Event([2,4,6,8,10,12,14,16,18,20],box21)
print(evMul2)
evMul2.print_prob()

[('2',), ('4',), ('6',), ('8',), ('10',), ('12',), ('14',), ('16',), ('18',), ('20',)]: 0.47619047619
Probability: 0.47619047619 ( 10/21 )


### Event: the extracted ball is multiple of 3

In [49]:
evMul3 = Event([3,6,9,12,15,18,21],box21)
print(evMul3)
evMul3.print_prob()

[('3',), ('6',), ('9',), ('12',), ('15',), ('18',), ('21',)]: 0.333333333333
Probability: 0.333333333333 ( 1/3 )


In [50]:
evMul2.is_independent(evMul3)

False

## Birthday problem

https://en.wikipedia.org/wiki/Birthday_problem

In [57]:
prob = 1
for k in range(23):
    comp = PExp(population={'C':1+k, 'nC':364-k})
    ev = Event([('nC')],comp)
    prob *= ev.prob
print(1-prob)

0.538344257915
